# Парсер страницы сайта.

## Часть 2. Загрузка HTML-кода из файла и работа с ним.

### 1. Подключение библиотек.

In [1]:
import bs4
import pandas as pd

### 2. Загрузка HTML-кода и получение BeautifulSoup для парсинга.

In [2]:
txt = open("soup.txt","r", encoding='utf-8').read()
b = bs4.BeautifulSoup(txt, 'html.parser')

### 3. Подбор тегов с нужной информацией в коде страницы. 

In [3]:
books = b.select('div.ml-sm-5 a h4')
autors = b.select('div.ml-sm-5 ul.my-2')
r_counts = b.select('div.rating-stars.d-none.d-sm-block.my-2 span.rates-count')
readings = b.select('ul.statistics-list li.reading')
planneds = b.select('ul.statistics-list li.planned')
reads = b.select('ul.statistics-list li.read')
not_reads = b.select('ul.statistics-list li.not-finish-read')
discuss = b.select('ul.statistics-list li.discuss')
cited = b.select('ul.statistics-list li.cited')
genres = b.select('div.col-sm.ml-sm-5')

###  4. Получение текстовых данных и их запись в таблицу.

In [4]:
df = pd.DataFrame()

for i in range(len(books)):
    df.loc[i, 'book'] = books[i].text
    for j in range(len(autors[i].select('a'))):
        df.loc[i, 'autor_' + str(j+1)] = autors[i].select('a')[j].text
    df.loc[i, 'rates_count'] = int(r_counts[i].text)
    df.loc[i, 'reading'] = int(readings[i].text)
    df.loc[i, 'planned'] = int(planneds[i].text)
    df.loc[i, 'read'] = int(reads[i].text)
    df.loc[i, 'not_read'] = int(not_reads[i].text)
    df.loc[i, 'discuss'] = int(discuss[i].text)
    df.loc[i, 'cited'] = int(cited[i].text)
    for j in range(len(genres[i].select('li.genre a'))):
        df.loc[i, 'genre_' + str(j+1)] = genres[i].select('li.genre a')[j].text

### 5.  Предобработка данных в датафрейме.

In [5]:
df = df[['book', 'autor_1', 'autor_2', 'genre_1', 'genre_2', 'genre_3', 'rates_count', 'reading', 'planned', 'read', 'not_read', 'discuss', 'cited']]
list_int = ['rates_count', 'reading', 'planned', 'read', 'not_read', 'discuss', 'cited']

for r in list_int:
    df[r] = df[r].astype(int)

list_str = ['autor_2', 'genre_1', 'genre_2', 'genre_3']

for r in list_str:
    df[r].fillna('отсутствует', inplace=True)
    
display(df)
df.info()

,book,autor_1,autor_2,genre_1,genre_2,genre_3,rates_count,reading,planned,read,not_read,discuss,cited
0,Время и снова время,Бен Элтон,отсутствует,Фантастика,Проза,отсутствует,14,1,31,25,0,3,0
1,Жена путешественника во времени,Одри Ниффенеггер,отсутствует,Любовные романы,Проза,отсутствует,47,4,45,72,0,4,15
2,Детская книга,Борис Акунин,отсутствует,Фантастика,Внеклассное чтение,отсутствует,23,0,9,53,0,3,0
3,11/22/63,Стивен Кинг,отсутствует,Фантастика,Детективы,Проза,267,49,382,488,6,5,13
4,Часодеи. Часовой ключ,Наталья Щерба,отсутствует,Детские приключенческие книги,отсутствует,отсутствует,19,1,21,40,0,5,1
5,Чужестранка,Диана Гэблдон,отсутствует,Любовные романы,Проза,отсутствует,7,0,16,20,0,1,0
6,Янки из Коннектикута при дворе короля Артура,Марк Твен,отсутствует,Фантастика,Приключения,Проза,10,2,10,15,0,0,63
7,Marvel. 1602,Нил Гейман,отсутствует,Комиксы и манга,отсутствует,отсутствует,4,0,8,6,0,1,0
8,Таймлесс. Рубиновая книга,Керстин Гир,отсутствует,отсутствует,отсутствует,отсутствует,39,3,13,102,0,3,8
9,Время всегда хорошее,Андрей Жвалевский,Евгения Пастернак,Детская литература,Внеклассное чтение,Книги для подростков,12,2,9,22,0,2,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   book         10 non-null     object
 1   autor_1      10 non-null     object
 2   autor_2      10 non-null     object
 3   genre_1      10 non-null     object
 4   genre_2      10 non-null     object
 5   genre_3      10 non-null     object
 6   rates_count  10 non-null     int32 
 7   reading      10 non-null     int32 
 8   planned      10 non-null     int32 
 9   read         10 non-null     int32 
 10  not_read     10 non-null     int32 
 11  discuss      10 non-null     int32 
 12  cited        10 non-null     int32 
dtypes: int32(7), object(6)
memory usage: 1.1+ KB


### 6. Запись полученной таблицы в exel и csv файлы на локальном диске. 

In [6]:
df.to_csv('books.csv', encoding='utf-8', index=False)

writer = pd.ExcelWriter('books.xlsx')
df.to_excel(writer)
writer.save()